In [80]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import numbers
import decimal
import scipy.stats as ss
import matplotlib.pyplot as plt
from statistics import stdev
from statistics import mean
import time
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
stop_words=set(stopwords.words("english"))
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
import xgboost as xgb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to /Users/faizank/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def getScoretWithModel(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

In [3]:
redditDataTrain = pd.read_csv("data/reddit_train.csv") #, sep="\n", header=None) 
redditDataTest = pd.read_csv("data/reddit_test.csv") # sep="\n", header=None)
redditDataTrain

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny
...,...,...,...
69995,69995,"Thank you, you confirm Spain does have nice pe...",europe
69996,69996,Imagine how many he would have killed with a r...,leagueoflegends
69997,69997,Yes. Only. As in the guy I was replying to was...,canada
69998,69998,Looking for something light-hearted or has a v...,anime


In [4]:
commentsTrain = redditDataTrain.iloc[:,1]
subredditsTrain = redditDataTrain.iloc[:,-1]
commentsTest = redditDataTest.iloc[:,1]

In [5]:
from models.naive_bayes import MultiNomialNB as CustomNaiveBayes

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
cv = CountVectorizer()
lr = LogisticRegression()
multiNB = MultinomialNB()
dtc = tree.DecisionTreeClassifier()
kf = StratifiedKFold(n_splits=5)
cnb = CustomNaiveBayes()

smoothing factor 0.01


### Part 2: Experimentation

In [7]:
kf = StratifiedKFold(n_splits=4)
kf = KFold(n_splits=4)

In [8]:
# GX_train, GX_test, Gy_train, Gy_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2)

In [12]:
GX_train = commentsTrain[commentsTrain.index < np.percentile(commentsTrain.index, 80)].sort_index()
Gy_train = subredditsTrain[subredditsTrain.index < np.percentile(subredditsTrain.index, 80)].sort_index()
GX_test = commentsTrain[commentsTrain.index > np.percentile(commentsTrain.index, 80)].sort_index()
Gy_test = subredditsTrain[subredditsTrain.index > np.percentile(subredditsTrain.index, 80)].sort_index()

### so I am splitting the global train data into Gtest and Gtrain. Then I use k-fold on Gtrain. in the k-fold I select the model that gives the best accuracy. Then I use that model, to predict the global thing. Ideally I should be using k-fold for validating which theory about the data is correct. Then train the validation theory, using the entire Gtrain and predict Gtest.

### get avg accuracy

In [12]:
kf = KFold(n_splits=4)

In [140]:
def get_avg_acc_for_model(model, **kwargs):
    results = []
    for train_index, test_index in kf.split(GX_train, Gy_train):
        x_train, x_test, y_train, y_test = GX_train[train_index], GX_train[test_index], Gy_train[train_index], Gy_train[test_index]
        redditDataTrainTF = tfidf.fit_transform(x_train)
        redditDataTestTF = tfidf.transform(x_test)
        redditDataTrainTF.toarray()
        clf = model(**kwargs)
        results.append(getScoretWithModel(clf, redditDataTrainTF, redditDataTestTF, y_train, y_test))
    avg_acc = sum(results)/len(results)
    return avg_acc

In [141]:
models = [LogisticRegression, MultinomialNB, CustomNaiveBayes]

In [142]:
for model in models:
    print("test for model", model)
    print(get_avg_acc_for_model(model))

test for model <class 'sklearn.linear_model.logistic.LogisticRegression'>


/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to si

0.5355714285714286
test for model <class 'sklearn.naive_bayes.MultinomialNB'>
0.5502857142857143
test for model <class 'models.naive_bayes.MultiNomialNB'>
smoothing factor 0.01
smoothing factor 0.01
smoothing factor 0.01
smoothing factor 0.01
0.5352321428571429


#### look for the best smoothing factor

In [1501]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.001))

smoothing factor 0.001
smoothing factor 0.001
smoothing factor 0.001
smoothing factor 0.001
smoothing factor 0.001
0.519267181569372


In [1506]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.0001))

smoothing factor 0.0001
smoothing factor 0.0001
smoothing factor 0.0001
smoothing factor 0.0001
0.5066964285714286


In [1507]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.00001))

smoothing factor 1e-05
smoothing factor 1e-05
smoothing factor 1e-05
smoothing factor 1e-05
0.49926785714285715


In [1508]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.01))

smoothing factor 0.01
smoothing factor 0.01
smoothing factor 0.01
smoothing factor 0.01
0.5352321428571429


In [1509]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.02))

smoothing factor 0.02
smoothing factor 0.02
smoothing factor 0.02
smoothing factor 0.02
0.5413928571428571


In [1510]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.03))

smoothing factor 0.03
smoothing factor 0.03
smoothing factor 0.03
smoothing factor 0.03
0.5443214285714285


In [1512]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5529821428571429


In [1513]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=1))

smoothing factor 1
smoothing factor 1
smoothing factor 1
smoothing factor 1
0.4699285714285714


#### standard tfidf accuracy

In [1521]:
tfidf = TfidfVectorizer(stop_words='english')
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5529821428571429


In [1522]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5532678571428571


In [1523]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii')
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5534285714285714


In [13]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1))
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5534285714285714


#### test lowercase

In [16]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5534285714285714


##### test min_df

In [11]:
tfidf = TfidfVectorizer(min_df=3, stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5420178571428571


#### test tokenizer, best so far

In [15]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5556607142857143


#### test stop words

In [16]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(sublinear_tf=True, stop_words=stop_words, strip_accents='ascii', ngram_range=(1, 1), lowercase=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5536964285714285


#### stemming

In [23]:
def stem_tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

In [11]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = stem_tokenize)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

NameError: name 'stem_tokenize' is not defined

In [58]:
Gx_train, Gx_test, y_train, y_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2, random_state=4)

In [ ]:
tfidf = TfidfVectorizer(stop_words="english", smooth_idf=True, sublinear_tf=True, use_idf=True, strip_accents='unicode',min_df=1, tokenizer = stem_tokenize) #, max_df=0.95 )
GX_train_idf_tokenized = tfidf.fit_transform(Gx_train)
GX_test_idf_tokenized = tfidf.transform(Gx_test)

#### retrain with the entire model and test final accuracy

In [26]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/faizank/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [42]:
sent="this is a test!."
sent_tokens = word_tokenize(sent)
nltk.pos_tag(sent)

[('t', 'NN'),
 ('h', 'NN'),
 ('i', 'JJ'),
 ('s', 'VBP'),
 (' ', 'NN'),
 ('i', 'NN'),
 ('s', 'VBP'),
 (' ', 'PDT'),
 ('a', 'DT'),
 (' ', 'JJ'),
 ('t', 'NN'),
 ('e', 'NN'),
 ('s', 'JJ'),
 ('t', 'NN'),
 ('!', '.'),
 ('.', '.')]

In [13]:
Gx_train, Gx_test, y_train, y_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2, random_state=4)

In [37]:
def stem_tokenize(text):
    tokens = word_tokenize(text)
    return nltk.pos_tag(tokens)

In [51]:
def stop_words_tokenizer(stop_words):
    tokens = word_tokenize(stop_words)
    return nltk.pos_tag(tokens)

In [54]:
stop_words = stopwords.words("english")
nltk.pos_tag(stop_words)

[('i', 'IN'),
 ('me', 'PRP'),
 ('my', 'PRP$'),
 ('myself', 'PRP'),
 ('we', 'PRP'),
 ('our', 'PRP$'),
 ('ours', 'PRP'),
 ('ourselves', 'PRP'),
 ('you', 'PRP'),
 ("you're", 'VBP'),
 ("you've", 'JJ'),
 ("you'll", 'NN'),
 ("you'd", 'VB'),
 ('your', 'PRP$'),
 ('yours', 'NNS'),
 ('yourself', 'PRP'),
 ('yourselves', 'VBZ'),
 ('he', 'PRP'),
 ('him', 'PRP'),
 ('his', 'PRP$'),
 ('himself', 'PRP'),
 ('she', 'PRP'),
 ("she's", 'VBD'),
 ('her', 'PRP'),
 ('hers', 'NNS'),
 ('herself', 'VBD'),
 ('it', 'PRP'),
 ("it's", 'VBZ'),
 ('its', 'PRP$'),
 ('itself', 'PRP'),
 ('they', 'PRP'),
 ('them', 'PRP'),
 ('their', 'PRP$'),
 ('theirs', 'JJ'),
 ('themselves', 'PRP'),
 ('what', 'WP'),
 ('which', 'WDT'),
 ('who', 'WP'),
 ('whom', 'WP'),
 ('this', 'DT'),
 ('that', 'WDT'),
 ("that'll", 'VBZ'),
 ('these', 'DT'),
 ('those', 'DT'),
 ('am', 'VBP'),
 ('is', 'VBZ'),
 ('are', 'VBP'),
 ('was', 'VBD'),
 ('were', 'VBD'),
 ('be', 'VB'),
 ('been', 'VBN'),
 ('being', 'VBG'),
 ('have', 'VB'),
 ('has', 'VBZ'),
 ('had', 'VBN')

In [71]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', strip_accents='unicode', max_df=0.9) #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = MultinomialNB(alpha=0.3, fit_prior=False)
# clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

0.573


In [77]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', strip_accents='unicode') #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = xgb.XGBClassifier()
# clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

0.43307142857142855


In [79]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', strip_accents='unicode') #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = SVC()
# clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.048357142857142855


In [116]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words="english", strip_accents='unicode', max_df=0.1) #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = RandomForestClassifier(n_estimators = 10)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

0.40185714285714286


In [117]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words="english", strip_accents='unicode', max_df=0.1) #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = RandomForestClassifier(max_depth=5, n_estimators = 10)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

0.11571428571428571


In [118]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words="english", strip_accents='unicode', max_df=0.1) #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = RandomForestClassifier(max_depth=10, n_estimators = 10)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

0.16085714285714287


In [ ]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words="english", strip_accents='unicode', max_df=0.1) #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = RandomForestClassifier(max_depth=4000, n_estimators = 1000)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

In [ ]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words="english", strip_accents='unicode', max_df=0.1) #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = RandomForestClassifier(max_depth=1000, n_estimators = 4000)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

In [ ]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words="english", strip_accents='unicode', max_df=0.1) #, max_df=0.95 )
GX_train_idf = tfidf.fit_transform(Gx_train)
GX_test_idf = tfidf.transform(Gx_test)
clf = RandomForestClassifier(max_depth=9000, n_estimators = 9000)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
print(final_acc)

In [53]:
tfidf = TfidfVectorizer(stop_words="english", smooth_idf=True, sublinear_tf=True, use_idf=True, strip_accents='unicode',min_df=1) #, max_df=0.95 )
multiNB = MultinomialNB(alpha=0.3)

def getScoretWithModel(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)



results = []
# for train_index, test_index in kf.split(commentsTrain, subredditsTrain):
x_train, x_test, y_train, y_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2, random_state=4)
redditDataTrainTF = tfidf.fit_transform(x_train)
redditDataTestTF = tfidf.transform(x_test)
    # redditDataTrainTF.toarray()
    # print(redditDataTrainTF.shape)
    # feature_scores = mutual_info_classif(redditDataTrainTF, y_train1)
    # print(feature_scores)
print(getScoretWithModel(multiNB, redditDataTrainTF, redditDataTestTF, y_train, y_test))
#     print(results)
# print(sum(results)/len(results))

0.5729285714285715


In [72]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', strip_accents='unicode', max_df=0.9) #, max_df=0.95 )
# Gx_train, Gx_test, y_train, y_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2, random_state=4)
# GX_train_idf = tfidf.fit_transform(Gx_train)
# GX_test_idf = tfidf.transform(Gx_test)
# final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
# print(final_acc)

# tfidf = TfidfVectorizer(stop_words='english', strip_accents='unicode', max_df=0.1) #, max_df=0.95 )
# GX_train_idf = tfidf.fit_transform(Gx_train)
# GX_test_idf = tfidf.transform(Gx_test)
# clf = MultinomialNB(alpha=0.3, fit_prior=False)
# # clf = CustomNaiveBayes(smoothing_factor=0.1)
# final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, y_train, y_test)
# print(final_acc)

df = pd.read_csv("data/reddit_test.csv", )
actual_Xtest = df.comments

commentsTrainidf = tfidf.fit_transform(commentsTrain)
actual_Xtestidf = tfidf.transform(actual_Xtest)

clf = MultinomialNB(alpha=0.3, fit_prior=False)
clf.fit(commentsTrainidf, subredditsTrain)

pred = clf.predict(actual_Xtestidf)
# print(pred)
pred = pd.DataFrame(pred, columns=['Category']).to_csv("results/testResults6.csv")

In [ ]:
df = pd.read_csv("data/reddit_test.csv", )
actual_Xtest = df.comments

commentsTrainidf = tfidf.fit_transform(commentsTrain)
actual_Xtestidf = tfidf.transform(actual_Xtest)

clf = CustomNaiveBayes(smoothing_factor=0.1)
clf.fit(commentsTrainidf, subredditsTrain)

pred = clf.predict(actual_Xtestidf)
# print(pred)
pred = pd.DataFrame(pred, columns=['Category']).to_csv("results/testResults3.csv")

In [97]:
df = pd.read_csv("data/test1.csv", )
df = df[['comments','subreddits']]
df.index.name = 'id'

In [102]:
total_Xtrain = commentsTrain
total_Xtrain_idf = tfidf.fit_transform(total_Xtrain)

total_ytrain = subredditsTrain

actual_Xtest = df.comments
actual_Xtest_idf = tfidf.transform(actual_Xtest)

actual_ytest = df.subreddits


clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, actual_Xtest_idf, Gy_train, actual_ytest)
print(final_acc)

smoothing factor 0.1


ValueError: dimension mismatch

In [9]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

smoothing factor 0.1
0.5657857142857143


In [27]:
def stem_tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = stem_tokenize)
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = MultinomialNB(alpha=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'ha', 'hi', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'thi', 'veri', 'wa', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


0.5619285714285714


In [28]:
tfidf = TfidfVectorizer(stop_words='english')
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

smoothing factor 0.1
0.5658571428571428


In [1486]:
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = tree.DecisionTreeClassifier()
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

0.32971428571428574


In [18]:
df = pd.read_csv("data/test1.csv", )
df = df[['comments','subreddits']]
df.index.name = 'id'

In [ ]:
actual_testX = df.comments
actual_testy = df.subreddits